# 2023 NFL Game Predictor

Import libraries

In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

Import data

In [2]:
import pandas as pd
import numpy as np

# Load the dataset for the 2023 season that include a "HomeWon" column.
# This column indicates if the home team won the game (1 for win, 0 for loss).
data_2023_updated = pd.read_csv("/Users/andynguyen/Desktop/nfl-game-predictor/BEST_2023_nfl_data_play_with_scores_and_HomeWon.csv")

# Load the dataset containing the upcoming games schedule.
upcoming_games = pd.read_csv("/Users/andynguyen/Desktop/nfl-game-predictor/BEST New_Upcoming_Schedule.csv")

# Load 2023 season data into a single DataFrame.
all_data = data_2023_updated

# Calculate Offensive Features
Average Points, Average Points Allowed, Overall Average, and Win Rate

In [3]:
# 1. Average Points Scored
# Calculate the average points scored by the home and visitor teams.
avg_points_scored_home = all_data.groupby('Home')['HomeScore'].mean()
avg_points_scored_visitor = all_data.groupby('Visitor')['VisitorScore'].mean()

# 2. Average Points Allowed
# Calculate the average points allowed by the home and visitor teams.
avg_points_allowed_home = all_data.groupby('Home')['VisitorScore'].mean()
avg_points_allowed_visitor = all_data.groupby('Visitor')['HomeScore'].mean()

# Calculate the overall average points scored and allowed by combining the home and visitor pages
overall_avg_points_scored = (avg_points_scored_home + avg_points_scored_visitor) / 2
overall_avg_points_allowed = (avg_points_allowed_home + avg_points_allowed_visitor) / 2

# 3. Win Rate
# Calculate the total number of wins for home and visitor teams.
home_wins = all_data.groupby('Home')['HomeWon'].sum()
visitor_wins = all_data.groupby('Visitor')['HomeWon'].apply(lambda x: len(x) - x.sum())

# Depreciated code
# visitor_wins = all_data.groupby('Visitor').apply(lambda x: len(x) - x['HomeWon'].sum())

# Calculate the total number of games played by each team as a home and visitor.
total_games_home = all_data['Home'].value_counts()
total_games_visitor = all_data['Visitor'].value_counts()

# Calculate the overall number of wins and total games played by each team.
overall_wins = home_wins + visitor_wins
total_games = total_games_home + total_games_visitor

# Calculate the overall number of wins and total games played by each team.
win_rate = overall_wins / total_games

Create new data frame to store the features for each team

In [4]:
team_features = pd.DataFrame({
    'AvgPointsScored': overall_avg_points_scored,
    'AvgPointsAllowed': overall_avg_points_allowed,
    'WinRate': win_rate
})

# Reset the index of the team features DataFrame and rename the index column to "Team".
team_features.reset_index(inplace=True)
team_features.rename(columns={'Home': 'Team'}, inplace=True)

# Display the first few rows of the team_features DataFrame.
team_features.head()

,Team,AvgPointsScored,AvgPointsAllowed,WinRate
0,ARI,22.000000,25.355769,0.260526
1,ATL,15.492551,19.294365,0.500675
2,BAL,24.633973,14.814507,0.713392
3,BUF,34.659834,13.942320,0.739255
4,CAR,17.259582,25.820934,0.000000


In [5]:
# Examining the columns and a few rows of the datasets to ensure correct structure
upcoming_games.head()

,Home,Visitor
0,CHI,WAS
1,HOU,ATL
2,CAR,DET
3,NO,NE
4,CIN,ARI


# Calculate Defensive Features
Average Points Defended, Average Conceded Plays, Average Forced Turnovers

In [6]:
# 1. Average points defended:
# This metric is essentially the same as AvgPointsAllowed, which we already computed in previous steps so we won't recompute it here.

# 2. Average conceded plays:
# A play is considered successful for the offense if it results in a touchdown or doesn't result in a turnover.
# Create a new column 'SuccessfulPlay' in the all_data DataFrame to represent this.
all_data['SuccessfulPlay'] = all_data['IsTouchdown'] | (~all_data['IsInterception'] & ~all_data['IsFumble'])

# Calculate the average rate of successful plays conceded when playing at home
avg_conceded_plays_home = all_data.groupby('Home')['SuccessfulPlay'].mean()

# Calculate the average rate of successful plays conceded when playing as a visitor.
avg_conceded_plays_visitor = all_data.groupby('Visitor')['SuccessfulPlay'].mean()

# Calculate the overall average rate of successful plays conceded when playing as a visitor.
overall_avg_conceded_plays = (avg_conceded_plays_home + avg_conceded_plays_visitor) / 2

# 3. Average forced turnovers:
# Create a new column 'Turnover' that indicates if a play resulted in a turnover (either interception or fumble).
all_data['Turnover'] = all_data['IsInterception'] | all_data['IsFumble']

# Calculate the average rate of turnovers forced when playing at home.
avg_forced_turnovers_home = all_data.groupby('Home')['Turnover'].mean()

# Calculate the average rate of turnovers forced when playing as a visitor.
avg_forced_turnovers_visitor = all_data.groupby('Visitor')['Turnover'].mean()

# Calculate the overall average rate of turnovers forced for each team.
overall_avg_forced_turnovers = (avg_forced_turnovers_home + avg_forced_turnovers_visitor) / 2

Create new DataFrame to store the defensive features for each team.

In [7]:
team_features_defensive = pd.DataFrame({
    'Team': team_features['Team'].values,
    'AvgPointsDefended': team_features['AvgPointsAllowed'].values,
    'AvgConcededPlays': overall_avg_conceded_plays.values,
    'AvgForcedTurnovers': overall_avg_forced_turnovers.values
})

# Combine offensive and defensive features
Merge the defensive features with the original team features to create a combined DataFrame.

In [8]:
# Merge the defensive features with the original team features to create a combined DataFrame.
team_features_combined = team_features.merge(team_features_defensive, on='Team')

# Display the first few rows of the combined team features DataFrame.
team_features_combined.head()

,Team,AvgPointsScored,AvgPointsAllowed,WinRate,AvgPointsDefended,AvgConcededPlays,AvgForcedTurnovers
0,ARI,22.000000,25.355769,0.260526,25.355769,-1.011919,0.013292
1,ATL,15.492551,19.294365,0.500675,19.294365,-1.017538,0.018890
2,BAL,24.633973,14.814507,0.713392,14.814507,-1.021287,0.021287
3,BUF,34.659834,13.942320,0.739255,13.942320,-1.030072,0.031484
4,CAR,17.259582,25.820934,0.000000,25.820934,-1.018031,0.020741


# Additional offesnive features
Average Yards per Play, Average Total Yards, Average Pass Completion Rate, Average Touchdowns per Game, Average Rush Success Rate

In [19]:
# Calculate additional offensive features

# 1. Average yards per play
avg_yards_per_play_home = all_data.groupby('Home')['Yards'].mean()
avg_yards_per_play_visitor = all_data.groupby('Visitor')['Yards'].mean()
overall_avg_yards_per_play = (avg_yards_per_play_home + avg_yards_per_play_visitor) / 2

# 3. Average pass completoon rate
avg_pass_completion_rate_home = 1 - all_data.groupby('Home')['IsIncomplete'].mean()
avg_pass_completion_rate_visitor = 1 - all_data.groupby('Visitor')['IsIncomplete'].mean()
overall_avg_pass_completion_rate = (avg_pass_completion_rate_home + avg_pass_completion_rate_visitor) / 2
                                                    
# 2. Average total yards per game
total_yards_per_game_home = all_data.groupby(['SeasonYear', 'Home'])['Yards'].sum() / all_data.groupby(['SeasonYear', 'Home']).size()
total_yards_per_game_visitor = all_data.groupby(['SeasonYear', 'Visitor'])['Yards'].sum() / all_data.groupby(['SeasonYear', 'Visitor']).size()
overall_avg_yards_per_game = (total_yards_per_game_home + total_yards_per_game_visitor).groupby(level=1).mean()

# 4. Average touchdowns per game
avg_touchdowns_per_game_home = all_data.groupby(['SeasonYear', 'Home'])['IsTouchdown'].sum() / all_data.groupby(['SeasonYear', 'Home']).size()
avg_touchdowns_per_game_visitor = all_data.groupby(['SeasonYear', 'Visitor'])['IsTouchdown'].sum() / all_data.groupby(['SeasonYear', 'Visitor']).size()
overall_avg_touchdowns_per_game = (avg_touchdowns_per_game_home + avg_touchdowns_per_game_visitor).groupby(level=1).mean()

# 5. Avrage rush success rate
avg_rush_success_rate_home = all_data[all_data['IsRush'] == 1].groupby('Home')['Yards'].mean()
avg_rush_success_rate_visitor = all_data[all_data['IsRush'] == 1].groupby('Visitor')['Yards'].mean()
overall_avg_rush_success_rate = (avg_rush_success_rate_home + avg_rush_success_rate_visitor) / 2

Create dataframe for the new offensive features

In [20]:
new_offensive_features = pd.DataFrame({
    'Team': team_features_combined['Team'],
    'AvgYardsPerPlay': overall_avg_yards_per_play.values,
    'AvgYardsPerGame': overall_avg_yards_per_game.values,
    'AvgPassCompletionRate': overall_avg_pass_completion_rate.values,
    'AvgTouchdownsPerGame': overall_avg_touchdowns_per_game.values,
    'AvgRushSuccessRate': overall_avg_rush_success_rate.values
})

# Merge with the existing combined features
team_features_expanded = team_features_combined.merge(new_offensive_features, on='Team')

team_features_expanded.head()

,Team,AvgPointsScored,AvgPointsAllowed,WinRate,AvgPointsDefended,AvgConcededPlays,AvgForcedTurnovers,AvgYardsPerPlay,AvgYardsPerGame,AvgPassCompletionRate,AvgTouchdownsPerGame,AvgRushSuccessRate
0,ARI,22.000000,25.355769,0.260526,25.355769,-1.011919,0.013292,3.954684,7.909368,0.884338,0.057776,4.984774
1,ATL,15.492551,19.294365,0.500675,19.294365,-1.017538,0.018890,3.375155,6.750310,0.867746,0.040468,4.482234
2,BAL,24.633973,14.814507,0.713392,14.814507,-1.021287,0.021287,3.368954,6.737907,0.883340,0.040894,4.483082
3,BUF,34.659834,13.942320,0.739255,13.942320,-1.030072,0.031484,4.145710,8.291420,0.906960,0.066286,5.679096
4,CAR,17.259582,25.820934,0.000000,25.820934,-1.018031,0.020741,3.490008,6.980017,0.867477,0.049143,4.736291


# Additional Defensive Features
Average Yards allowed per Play, Average Total Yards allowed per Game, Average Pass Completion allowed Rate, Average Touchdowns allowed per Game, Average Rush Success allowed Rate

In [34]:
# 1. Average yards allowed per play
avg_yards_allowed_per_play_home = all_data.groupby('Home')['Yards'].mean()
avg_yards_allowed_per_play_visitor = all_data.groupby('Visitor')['Yards'].mean()
overall_avg_yards_allowed_per_play = (avg_yards_allowed_per_play_home + avg_yards_allowed_per_play_visitor) / 2

# 2. Average total yards
total_yards_allowed_per_game_home = all_data.groupby(['SeasonYear', 'Home'])['Yards'].sum() / all_data.groupby(['SeasonYear', 'Home']).size()
total_yards_allowed_per_game_visitor = all_data.groupby(['SeasonYear', 'Visitor'])['Yards'].sum() / all_data.groupby(['SeasonYear', 'Visitor']).size()
overall_avg_yards_allowed_per_game = (total_yards_allowed_per_game_home + total_yards_allowed_per_game_visitor).groupby(level=1).mean()

# 3. Average pass completion allowed rate
avg_pass_completion_allowed_rate_home = 1 - all_data.groupby('Home')['IsIncomplete'].mean()
avg_pass_completion_allowed_rate_visitor = 1 - all_data.groupby('Visitor')['IsIncomplete'].mean()
overall_avg_pass_completion_allowed_rate = (avg_pass_completion_allowed_rate_home + avg_pass_completion_allowed_rate_visitor) / 2

# 4. Average touchdowns allowed per game
avg_touchdowns_allowed_per_game_home = all_data.groupby(['SeasonYear', 'Home'])['IsTouchdown'].sum() / all_data.groupby(['SeasonYear', 'Home']).size()
avg_touchdowns_allowed_per_game_visitor = all_data.groupby(['SeasonYear', 'Visitor'])['IsTouchdown'].sum() / all_data.groupby(['SeasonYear', 'Visitor']).size()
overall_avg_touchdowns_allowed_per_game = (avg_touchdowns_allowed_per_game_home + avg_touchdowns_allowed_per_game_visitor).groupby(level=1).mean()

# 5. Average rush success allowed rate
avg_rush_success_allowed_rate_home = (
    all_data[all_data['IsRush'] == 1]
    .groupby('Home')['Yards']
    .mean()
)
avg_rush_success_allowed_rate_visitor = (
    all_data[all_data['IsRush'] == 1]
    .groupby('Visitor')['Yards']
    .mean()
)
overall_avg_rush_success_allowed_rate = (avg_rush_success_allowed_rate_home + avg_rush_success_allowed_rate_visitor) / 2


Create dataframe for the new defensive

In [36]:
new_defensive_features = pd.DataFrame({
    'Team': team_features_expanded['Team'],
    'AvgYardsAllowedPerPlay': overall_avg_yards_allowed_per_play.values,
    'AvgYardsAllowedPerGame': overall_avg_yards_allowed_per_game.values,
    'AvgPassCompletionAllowedRate': overall_avg_pass_completion_allowed_rate.values,
    'AvgTouchdownsAllowedPerGame': overall_avg_touchdowns_allowed_per_game.values,
    'AvgRushSuccessAllowedRate': overall_avg_rush_success_allowed_rate.values
})

# Merging with the existing combined features
team_features_complete = team_features_expanded.merge(new_defensive_features, on='Team')
                       
team_features_complete

,Team,AvgPointsScored,AvgPointsAllowed,WinRate,AvgPointsDefended,AvgConcededPlays,AvgForcedTurnovers,AvgYardsPerPlay,AvgYardsPerGame,AvgPassCompletionRate,AvgTouchdownsPerGame,AvgRushSuccessRate,AvgYardsAllowedPerPlay,AvgYardsAllowedPerGame,AvgPassCompletionAllowedRate,AvgTouchdownsAllowedPerGame,AvgRushSuccessAllowedRate
0,ARI,22.000000,25.355769,0.260526,25.355769,-1.011919,0.013292,3.954684,7.909368,0.884338,0.057776,4.984774,3.954684,7.909368,0.884338,0.057776,4.984774
1,ATL,15.492551,19.294365,0.500675,19.294365,-1.017538,0.018890,3.375155,6.750310,0.867746,0.040468,4.482234,3.375155,6.750310,0.867746,0.040468,4.482234
2,BAL,24.633973,14.814507,0.713392,14.814507,-1.021287,0.021287,3.368954,6.737907,0.883340,0.040894,4.483082,3.368954,6.737907,0.883340,0.040894,4.483082
3,BUF,34.659834,13.942320,0.739255,13.942320,-1.030072,0.031484,4.145710,8.291420,0.906960,0.066286,5.679096,4.145710,8.291420,0.906960,0.066286,5.679096
4,CAR,17.259582,25.820934,0.000000,25.820934,-1.018031,0.020741,3.490008,6.980017,0.867477,0.049143,4.736291,3.490008,6.980017,0.867477,0.049143,4.736291
5,CHI,18.665810,34.334604,0.000000,34.334604,-1.019208,0.023257,3.998670,7.997340,0.895361,0.076005,4.422727,3.998670,7.997340,0.895361,0.076005,4.422727
6,CIN,12.227273,23.409632,0.265583,23.409632,-1.019241,0.019241,3.427839,6.855679,0.852791,0.037541,4.911324,3.427839,6.855679,0.852791,0.037541,4.911324
7,CLE,19.919890,18.735727,0.484480,18.735727,-1.020230,0.025280,3.282982,6.565964,0.851052,0.041827,4.530804,3.282982,6.565964,0.851052,0.041827,4.530804
8,DAL,30.742164,10.539742,0.736352,10.539742,-1.021265,0.026639,3.528267,7.056535,0.877737,0.047950,4.580975,3.528267,7.056535,0.877737,0.047950,4.580975
9,DEN,25.289070,38.441368,0.239735,38.441368,-1.014579,0.015895,4.393474,8.786947,0.897912,0.092526,5.513577,4.393474,8.786947,0.897912,0.092526,5.513577


# Encoding upcoming games with features

In [41]:
# Reload the upcoming games data
upcoming_games = pd.read_csv('/Users/andynguyen/Desktop/nfl-game-predictor/BEST New_Upcoming_Schedule.csv')

# Feature encoding: mergin the upcoming games data with the team features data
upcoming_encoded_home = upcoming_games.merge(team_features_complete, left_on='Home', right_on='Team', how='left')
upcoming_encoded_both = upcoming_encoded_home.merge(team_features_complete, left_on='Visitor', right_on='Team', suffixes=('_Home', '_Visitor'), how='left')

upcoming_encoded_both

,Home,Visitor,Team_Home,AvgPointsScored_Home,AvgPointsAllowed_Home,WinRate_Home,AvgPointsDefended_Home,AvgConcededPlays_Home,AvgForcedTurnovers_Home,AvgYardsPerPlay_Home,...,AvgYardsPerPlay_Visitor,AvgYardsPerGame_Visitor,AvgPassCompletionRate_Visitor,AvgTouchdownsPerGame_Visitor,AvgRushSuccessRate_Visitor,AvgYardsAllowedPerPlay_Visitor,AvgYardsAllowedPerGame_Visitor,AvgPassCompletionAllowedRate_Visitor,AvgTouchdownsAllowedPerGame_Visitor,AvgRushSuccessAllowedRate_Visitor
0,CHI,WAS,CHI,18.665810,34.334604,0.000000,34.334604,-1.019208,0.023257,3.998670,...,3.558549,7.117099,0.873646,0.055276,4.664426,3.558549,7.117099,0.873646,0.055276,4.664426
1,HOU,ATL,HOU,23.762219,20.104762,0.483204,20.104762,-1.019390,0.020685,3.720154,...,3.375155,6.750310,0.867746,0.040468,4.482234,3.375155,6.750310,0.867746,0.040468,4.482234
2,CAR,DET,CAR,17.259582,25.820934,0.000000,25.820934,-1.018031,0.020741,3.490008,...,3.767808,7.535617,0.872035,0.059415,3.973810,3.767808,7.535617,0.872035,0.059415,3.973810
3,NO,NE,NO,15.474421,19.042558,0.496774,19.042558,-1.019450,0.019450,3.176105,...,3.517884,7.035767,0.872396,0.041135,3.929386,3.517884,7.035767,0.872396,0.041135,3.929386
4,CIN,ARI,CIN,12.227273,23.409632,0.265583,23.409632,-1.019241,0.019241,3.427839,...,3.954684,7.909368,0.884338,0.057776,4.984774,3.954684,7.909368,0.884338,0.057776,4.984774
5,NYJ,DEN,NYJ,13.541156,23.962347,0.239789,23.962347,-1.022022,0.024681,3.442188,...,4.393474,8.786947,0.897912,0.092526,5.513577,4.393474,8.786947,0.897912,0.092526,5.513577
6,DAL,SF,DAL,30.742164,10.539742,0.736352,10.539742,-1.021265,0.026639,3.528267,...,3.852099,7.704198,0.871341,0.053310,4.728202,3.852099,7.704198,0.871341,0.053310,4.728202
7,JAX,BUF,JAX,23.583784,20.864865,0.495352,20.864865,-1.023437,0.026863,3.756156,...,4.145710,8.291420,0.906960,0.066286,5.679096,4.145710,8.291420,0.906960,0.066286,5.679096
8,TEN,IND,TEN,18.124654,17.778393,0.500000,17.778393,-1.022161,0.022161,3.317175,...,3.630419,7.260839,0.882493,0.057721,4.104843,3.630419,7.260839,0.882493,0.057721,4.104843
9,NYG,MIA,NYG,10.835476,34.491003,0.345550,34.491003,-1.024163,0.026881,3.216616,...,5.139113,10.278227,0.896207,0.117335,6.231532,5.139113,10.278227,0.896207,0.117335,6.231532


# Calculate difference in features
This might be a more predictive representation

In [46]:
for col in ['AvgPointsScored', 'AvgPointsAllowed', 'WinRate', 'AvgPointsDefended', 'AvgConcededPlays', 'AvgForcedTurnovers',
            'AvgYardsPerPlay', 'AvgYardsPerGame', 'AvgPassCompletionRate', 'AvgTouchdownsPerGame', 'AvgRushSuccessRate',
            'AvgYardsAllowedPerPlay', 'AvgYardsAllowedPerGame', 'AvgPassCompletionAllowedRate', 'AvgTouchdownsAllowedPerGame', 'AvgRushSuccessAllowedRate']:
        upcoming_encoded_both[f'Diff_{col}'] = upcoming_encoded_both[f'{col}_Home'] - upcoming_encoded_both[f'{col}_Visitor']

# Selecting only the different columns and the teams for clarity
upcoming_encoded_final = upcoming_encoded_both[['Home', 'Visitor'] + [col for col in upcoming_encoded_both.columns if 'Diff_' in col]]

upcoming_encoded_final

,Home,Visitor,Diff_AvgPointsScored,Diff_AvgPointsAllowed,Diff_WinRate,Diff_AvgPointsDefended,Diff_AvgConcededPlays,Diff_AvgForcedTurnovers,Diff_AvgYardsPerPlay,Diff_AvgYardsPerGame,Diff_AvgPassCompletionRate,Diff_AvgTouchdownsPerGame,Diff_AvgRushSuccessRate,Diff_AvgYardsAllowedPerPlay,Diff_AvgYardsAllowedPerGame,Diff_AvgPassCompletionAllowedRate,Diff_AvgTouchdownsAllowedPerGame,Diff_AvgRushSuccessAllowedRate
0,CHI,WAS,-3.769975,4.578245,-0.507060,4.578245,-0.000762,0.000844,0.440121,0.880241,0.021714,0.020729,-0.241698,0.440121,0.880241,0.021714,0.020729,-0.241698
1,HOU,ATL,8.269668,0.810397,-0.017471,0.810397,-0.001852,0.001796,0.344999,0.689998,0.013390,0.016407,-0.663932,0.344999,0.689998,0.013390,0.016407,-0.663932
2,CAR,DET,-9.223491,5.070934,-0.748062,5.070934,-0.003808,0.003934,-0.277800,-0.555600,-0.004558,-0.010272,0.762481,-0.277800,-0.555600,-0.004558,-0.010272,0.762481
3,NO,NE,1.521934,-4.816605,0.235176,-4.816605,-0.002762,-0.001117,-0.341778,-0.683557,-0.019339,-0.010308,0.396533,-0.341778,-0.683557,-0.019339,-0.010308,0.396533
4,CIN,ARI,-9.772727,-1.946138,0.005056,-1.946138,-0.007323,0.005949,-0.526845,-1.053690,-0.031547,-0.020235,-0.073450,-0.526845,-1.053690,-0.031547,-0.020235,-0.073450
5,NYJ,DEN,-11.747914,-14.479022,0.000054,-14.479022,-0.007443,0.008787,-0.951286,-1.902571,-0.017719,-0.050169,-1.132587,-0.951286,-1.902571,-0.017719,-0.050169,-1.132587
6,DAL,SF,-0.453191,-3.947550,-0.263648,-3.947550,-0.005539,0.010913,-0.323832,-0.647663,0.006396,-0.005360,-0.147227,-0.323832,-0.647663,0.006396,-0.005360,-0.147227
7,JAX,BUF,-11.076051,6.922545,-0.243903,6.922545,0.006635,-0.004621,-0.389554,-0.779108,-0.028459,-0.011112,-1.762924,-0.389554,-0.779108,-0.028459,-0.011112,-1.762924
8,TEN,IND,-5.998614,-6.947875,-0.017365,-6.947875,0.000663,-0.003061,-0.313245,-0.626490,-0.005762,-0.024480,-0.419529,-0.313245,-0.626490,-0.005762,-0.024480,-0.419529
9,NYG,MIA,-37.730958,8.130863,-0.426175,8.130863,-0.001001,0.002844,-1.922497,-3.844995,-0.028556,-0.059313,-1.938757,-1.922497,-3.844995,-0.028556,-0.059313,-1.938757


# Why use the difference for features?

**Predictive Power**: Differences in team metrics might be more predictive of game outcomes than the absolute metrics of each team. For instance, if one team scores on average 10 points more than another team, this differential might be a stronger predictor of the game's outcome than knowing each team's average score in isolation.

**Simplification for Modeling**: By creating differential features, the model can focus on the relative strengths and weaknesses between the two teams, potentially simplifying the decision-making process of the model.

**Normalization**: Teams might play in different conditions, against different opponents, or might have had easy/hard schedules. Calculating the difference between their stats might normalize some of these external factors.


# Training Data Preparation

In [50]:
# Merge play-by-play data with team features for home teams
training_encoded_home = all_data.merge(team_features_complete, left_on='Home', right_on='Team', how='left')

# Merge the result with team features for visitor teams
training_encoded_both = training_encoded_home.merge(team_features_complete, left_on='Visitor', right_on='Team', suffixes=('_Home', '_Visitor'), how='left')

# Calculate the difference in features
for col in ['AvgPointsScored', 'AvgPointsAllowed', 'WinRate', 'AvgPointsDefended', 'AvgConcededPlays', 'AvgForcedTurnovers',
            'AvgYardsPerPlay', 'AvgYardsPerGame', 'AvgPassCompletionRate', 'AvgTouchdownsPerGame', 'AvgRushSuccessRate',
            'AvgYardsAllowedPerPlay', 'AvgYardsAllowedPerGame', 'AvgPassCompletionAllowedRate', 'AvgTouchdownsAllowedPerGame', 'AvgRushSuccessAllowedRate']:
        training_encoded_both[f'Diff_{col}'] = training_encoded_both[f'{col}_Home'] - training_encoded_both[f'{col}_Visitor']

# Filtering out the required columns
training_data = training_encoded_both[[col for col in training_encoded_both.columns if 'Diff_' in col]]
training_labels = training_encoded_both['HomeWon']

In [51]:
training_data.head()

,Diff_AvgPointsScored,Diff_AvgPointsAllowed,Diff_WinRate,Diff_AvgPointsDefended,Diff_AvgConcededPlays,Diff_AvgForcedTurnovers,Diff_AvgYardsPerPlay,Diff_AvgYardsPerGame,Diff_AvgPassCompletionRate,Diff_AvgTouchdownsPerGame,Diff_AvgRushSuccessRate,Diff_AvgYardsAllowedPerPlay,Diff_AvgYardsAllowedPerGame,Diff_AvgPassCompletionAllowedRate,Diff_AvgTouchdownsAllowedPerGame,Diff_AvgRushSuccessAllowedRate
0,11.278682,1.763204,0.260560,1.763204,0.002583,-0.001320,0.418240,0.836480,-0.011787,0.016709,0.249559,0.418240,0.836480,-0.011787,0.016709,0.249559
1,11.278682,1.763204,0.260560,1.763204,0.002583,-0.001320,0.418240,0.836480,-0.011787,0.016709,0.249559,0.418240,0.836480,-0.011787,0.016709,0.249559
2,11.278682,1.763204,0.260560,1.763204,0.002583,-0.001320,0.418240,0.836480,-0.011787,0.016709,0.249559,0.418240,0.836480,-0.011787,0.016709,0.249559
3,11.278682,1.763204,0.260560,1.763204,0.002583,-0.001320,0.418240,0.836480,-0.011787,0.016709,0.249559,0.418240,0.836480,-0.011787,0.016709,0.249559
4,16.789677,-13.319420,0.474754,-13.319420,-0.004577,0.006072,0.010384,0.020767,0.005341,0.006815,0.651589,0.010384,0.020767,0.005341,0.006815,0.651589


In [52]:
training_data.shape

(12009, 16)

In [53]:
training_labels.head()

0    True
1    True
2    True
3    True
4    True
Name: HomeWon, dtype: bool

# AI Model Training

In [61]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

# Initialize the logisitc regression model
logreg = LogisticRegression(max_iter=1000)

# Evaluate the model's performance using cross-validation
cross_val_scores = cross_val_score(logreg, training_data, training_labels, cv=5)

cross_val_scores_mean = cross_val_scores.mean()

cross_val_scores_mean

np.float64(0.75302023407538)

In [60]:
# Checking the shape of the training data
training_data.shape

(12009, 16)

You will get an error like this
"Error because of NaN values ValueError: Input contains NaN, infinity or a value too large for dtype('float64')."

Check how many rows are affected

In [64]:
# Checking for NaN values in the training data
nan_columns = training_data.columns[training_data.isna().any()].tolist()

# Displaying columns with NaN values and the number of NaN values in them
nan_counts = training_data[nan_columns].isna().sum()
nan_counts

Series([], dtype: float64)

The output indicates that there are 3,288 rows (entries) in our training data where the difference values for the specified columns are NaN. 

This suggests that for these rows, either the home or visitor team (or possibly both) didn't have corresponding data in our team features dataset, leading to the NaN values when calculating the difference.

It would require us to take a closer look at the data to find out why this is. It could potentially be games that were called off, or certain plays that have NaN values, or something else.

However, the rows with NaN values constitute only about 6.5% of our total training data.

Excluding this proportion of the data might result in a slight reduction in the training data's robustness and diversity. However, given that it's a relatively small fraction of the total, excluding these rows likely won't have a major detrimental effect on the model's training and performance.

And since it may take significantly more time to solve this issue, I decided that I would remove these, leave it as it is, and live with the results. For a more important project where the highest accuracy is absolutely crucial, you would likely need to investigate this further.

In [65]:
# Exclude rows with NaN values from the training data and labels
training_data_cleaned = training_data.dropna()
training_labels_cleaned = training_labels.loc[training_data_cleaned.index]

# Checking the shape of the cleaned data
training_data_cleaned.shape, training_labels_cleaned.shape

((12009, 16), (12009,))

The rows with NaN values have been successfully excluded. Now, our cleaned training data consists of 47,319 rows (entries) and 16 feature columns.

Let's try again now...

In [67]:
# Re-evaluate the model's performance using cross-validation on the cleaned data
cross_val_scores_cleaned = cross_val_score(logreg, training_data_cleaned, training_labels_cleaned, cv=5)

cross_val_scores_cleaned_mean = cross_val_scores_cleaned.mean()

cross_val_scores_cleaned_mean

np.float64(0.75302023407538)

Nice! So our model, after performing a 5 fold cross validation on our training data has achieved a 71% accuracy in predicting the winners of the games.

If you'd like more info on cross-validation, here is a gentle intro to it

The next step would be to train the model on the entire cleaned training dataset and then use it to predict the outcomes of the upcoming games.

# Train the Model on Entire cleaned Data

In [68]:
# Train the logistic regressio model on the entire cleaned training dataset
logreg.fit(training_data_cleaned, training_labels_cleaned)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,1000
,multi_class,'deprecated'


# Make Predictons on Upcoming Games
This data is for the upcoming NFL matchups for Week 5 in the 2023 season. Feel free to check after these games have happened to see how accurate the predictions were.

In [70]:
# Predict the probability of the home team winning for the upcoming games
upcoming_game_probabilities = logreg.predict_proba(upcoming_encoded_final[[col for col in upcoming_encoded_final.columns if 'Diff_' in col]])

upcoming_game_probabilities

array([[0.95763121, 0.04236879],
       [0.55390523, 0.44609477],
       [0.99491747, 0.00508253],
       [0.22328809, 0.77671191],
       [0.39716048, 0.60283952],
       [0.41028229, 0.58971771],
       [0.8987413 , 0.1012587 ],
       [0.61175983, 0.38824017],
       [0.46961497, 0.53038503],
       [0.75684827, 0.24315173],
       [0.39386212, 0.60613788],
       [0.03464301, 0.96535699],
       [0.04173709, 0.95826291],
       [0.22265833, 0.77734167]])

These numbers are the probabilities of each team winning the game

In [71]:
# Extract the probability that the home team will win (second column of the result)
upcoming_game_prob_home_win = upcoming_game_probabilities[:, 1]

# Add the predictions to the upcoming games dataframe
upcoming_encoded_final['HomeWinProbability'] = upcoming_game_prob_home_win

# Sort by the probability of the home team winning for better visualization
upcoming_predictions = upcoming_encoded_final[['Home', 'Visitor', 'HomeWinProbability']].sort_values(by='HomeWinProbability', ascending=False)

upcoming_predictions

/var/folders/30/hnnymx5s151fpvvdt1qkvp840000gn/T/ipykernel_89904/2773497604.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  upcoming_encoded_final['HomeWinProbability'] = upcoming_game_prob_home_win


,Home,Visitor,HomeWinProbability
11,PHI,LA,0.965357
12,KC,MIN,0.958263
13,GB,LV,0.777342
3,NO,NE,0.776712
10,BAL,PIT,0.606138
4,CIN,ARI,0.602840
5,NYJ,DEN,0.589718
8,TEN,IND,0.530385
1,HOU,ATL,0.446095
7,JAX,BUF,0.388240


# Flaws with this approach

Some of the flaws with this approach
- Doesn't account for injuries from least season to this one
- Doesn't account for team trades or hiring/firings
- Doesn't account for weather